### Bronze Layer: Ingest qualifying.json

Load raw JSON (multiline) from landing zone and save as parquet in bronze/raw layer.

In [1]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

/home/ricardo/Documentos/Projects/dab-ci-cd/dab_project/.venv/lib/python3.12/site-packages/databricks/sdk/_widgets/__init__.py:71: UserWarning: 
To use databricks widgets interactively in your notebook, please install databricks sdk using:
	pip install 'databricks-sdk[notebook]'
Falling back to default_value_only implementation for databricks widgets.
  warnings.warn(


In [2]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import current_timestamp, lit

In [3]:
from formula1.formula1_constants import landing_folder_path, raw_folder_path

##### Step 1 - Define schema and read JSON from landing zone

In [4]:
qualifying_schema = StructType(fields=[
    StructField("qualifyId", IntegerType(), False),
    StructField("raceId", IntegerType(), True),
    StructField("driverId", IntegerType(), True),
    StructField("constructorId", IntegerType(), True),
    StructField("number", IntegerType(), True),
    StructField("position", IntegerType(), True),
    StructField("q1", StringType(), True),
    StructField("q2", StringType(), True),
    StructField("q3", StringType(), True)
])

In [5]:
qualifying_df = spark.read \
    .option("multiLine", True) \
    .schema(qualifying_schema) \
    .json(f"{landing_folder_path}/qualifying/")

##### Step 2 - Add ingestion metadata

In [6]:
qualifying_df = qualifying_df \
    .withColumn("ingestion_date", current_timestamp()) \
    .withColumn("data_source", lit(v_data_source))

##### Step 3 - Write to bronze/raw layer as parquet

In [7]:
qualifying_df.write.mode("overwrite").parquet(f"{raw_folder_path}/qualifying")